# <center> Gender Biais detection</center>
## <center> Infer the Semmantic gender thanks to Word vectors in French  </center>
### <center> Quillivic Robin, 01/03/2019 </center>
In this notebook, we are trying to show that it's possible to infer the semantic gender based in French...

In [196]:
from gensim import *

from gensim.models import Word2Vec
import logging
import gensim
import os

import itertools

from gensim.test.utils import get_tmpfile
from gensim.models import KeyedVectors

import matplotlib.pyplot as plt
from random import *

import pandas as pd
import numpy as np
import math

from sklearn.decomposition import PCA
import lib
import fasttext
from gensim.models.wrappers import FastText
import random
import translate
from sklearn import svm

# Load the models

In [70]:
# Logging code taken from http://rare-technologies.com/word2vec-tutorial/
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

# Load pre-trained Word2Vec model.
model_cc  = gensim.models.KeyedVectors.load_word2vec_format(r'DATA\FR\cc.fr.300.vec', binary=False,limit=75000)
model_fr_wac  = gensim.models.KeyedVectors.load_word2vec_format(r'DATA\FR\frWac_no_postag_no_phrase_500_skip_cut100.bin', binary=True,limit=50000)
model_wiki  = gensim.models.KeyedVectors.load_word2vec_format(r'DATA\FR\wiki_fr.vec',binary=False,limit=50000)
model_wiki_lem  = gensim.models.KeyedVectors.load_word2vec_format('C:/Users/Robin/Documents/IMPACT/Notebooks/DATA/FR/ppmi_svd_fr/matrix.txt',binary=False,encoding='latin1')

2019-03-19 10:52:28,550 : INFO : loading projection weights from DATA\FR\cc.fr.300.vec
2019-03-19 10:52:55,085 : INFO : loaded (75000, 300) matrix from DATA\FR\cc.fr.300.vec
2019-03-19 10:52:55,104 : INFO : loading projection weights from DATA\FR\frWac_no_postag_no_phrase_500_skip_cut100.bin
2019-03-19 10:52:56,203 : INFO : loaded (50000, 500) matrix from DATA\FR\frWac_no_postag_no_phrase_500_skip_cut100.bin
2019-03-19 10:52:56,226 : INFO : loading projection weights from DATA\FR\wiki_fr.vec
2019-03-19 10:53:13,437 : INFO : loaded (50000, 300) matrix from DATA\FR\wiki_fr.vec
2019-03-19 10:53:13,455 : INFO : loading projection weights from C:/Users/Robin/Documents/IMPACT/Notebooks/DATA/FR/ppmi_svd_fr/matrix.txt
2019-03-19 10:53:57,756 : INFO : loaded (53376, 500) matrix from C:/Users/Robin/Documents/IMPACT/Notebooks/DATA/FR/ppmi_svd_fr/matrix.txt


# Load the labels
How to identify the semantic gender in French:
* Manually : We took each word of the dict and we attribute m, f or neutral semantic gender.
* First idea : Using English translation. We use the gender in English as the semantic gender in French.
* Second idea:  use key word in the definition to infer the semantic gender
* Third idea : use small number  gender specific words an try tio generelize the classifier. 



In [148]:
#Load lexique in a Dataframe
df_fr = pd.read_excel(r'DATA\FR\Lexique382.xlsx',encoding='latin1')

In [54]:
df_fr_2 = pd.read_excel(r'DATA\FR\DEM-1_0.xlsx',enconding='utf8')
#http://rali.iro.umontreal.ca/rali/?q=fr/demn

In [60]:
def gender(x):
    x =str(x)
    if ('femme' in x) and ('homme' in x):
        return(0)
    elif ('femme' in x ) or ('femelle' in x ):
        return(-1)
    elif ('homme' in x) or ('male' in x):
        return(1)
    else :
        return(0)
    
    
    
    
df_fr_2['Genre'] = df_fr_2['SENS'].apply(gender)

# 3rd idea

In [192]:
gender_specific= ['femme','femelle','soeur', 'homme','frère','male','actrice','acteur', 'mariée','marie',
                       'époux','épouse','fille','garçon','maman','papa','mère','père','fille','fils', 'mâle',
                       'reine','roi','tante','oncle','grand-père','grand-mère','marraine','parrain','cavalier',
                       'cavalière','neveu','nièce','prince','princesse','sein','poitrine','pénis', 'veuf','veuve',
                       'brebis', 'bélier','chèvre', 'bouc','biche', 'cerf','chatte', 'chat', 'jument', 'cheval',
                       'chienne', 'chien', 'poule', 'coq','louve', 'loup','rate', 'rat','vache', 'taureau', 'âne', 'éléphant',
                       'hérisson', 'buffle', 'daim','cane', 'canard',
                       'dinde', 'chameau', 'chevreuil', 'crapaud',
                       'héron','oie', 'lapin','lièvre', 'tigre',
                       'lion', 'merle', 'ours', 'cochon', 'sanglier']

In [193]:
Gender_Neutral = ['motivation','apéritif', 'amené', 'images', 'Lejeune','affirmative','désolation',
                   'forumVous','fastidieuse','regarderai','permettra','Publié','Officiels','épuisés','froids',
                   'gouverné','transforma','styliste','appétit','grammaticale','Hospital', 'Hygiène','supérieur',
                   'défection','remanié', 'déplacements','Cirque','dispositif','continuum','Blu-Ray','chiites',
                   'reglage','Haha','forfaits','injure','Murcie','dérangé','Clio64','Scandinave','lactose',
                   'festifs','tnt','quadrangulaire','Vitrolles','clans','opale','Pulse','sérotonine','dépend',
                   'réglementée','Misty','96','tartine','crochets','Armin','câlins','Rhythm','Sol','rotations',
                   'Chirurgien','pressé','démoniaque','voïvodie','élire','détectés','Ji','oeuf','VM','Liner',
                   'gel','164','irritant','étoffes','od','infographie','Unix','Ft','perd','intrusions','Navigator',
                   'réservoir','Colombien','ou','af','déplorable','Offices','prodigués','message','kilogrammes',
                   'étrusques','raisonner','enrobé','boutures','poire','erre','forets','Ö','ortho','siren','tango',
                   'signale','pavée','référence','viticulteurs','tjs','brouillés','Rasage','example','CIR',
                   'confisque','astuce','Disposez','117','Personnages','AUTOMOBILE', 'TLS','entreprises','Favoris',
                   '149','Bir','est','abuse','schémas','abstenir','chemise','visite','troisieme','interesse',
                   'continus','saisi','apocalyptique','investisseurs','Andrée','Cherbourg','verse','appelle',
                   'déjeuners','rade','évangélisation','avançant','affriolantes','Perche','ovin',
                   'Rhénanie-Palatinat','Montes','rencontre','localisation','affiche','croissance','soumission',
                   'construction','déloger', 'caresse','stagnation','désinfecter','exportés','modification',
                   'Simulation','gente','artisan','déployés','ignorez','quantitatifs','cabane','disjoncteurs',
                   'patine','constitue','BIENVENUE','tendre','Conservateur','orne','dicter','Joyeuses','fêtera',
                   'viables','trial','potentiel','Crème','chimère','éditorialiste','Aznavour','orage','Paysagiste',
                   'cruel','plages','Solidarités','Braz','littorale','grand-chose','constituant','représentations',
                   'Théorie','tringle','drame','concurrentiel','tempe','pilotis','désactivation','destroyer','internet',
                   'outils','caser','Photocopieur','Demandeurs','Bruxelles','alpins','morphine','discutées','Rosette',
                   'néfastes','Câbles','fraîches','Absent','juridictionnelle','Porcelaine','terminus','nacelle','Idéal',
                   'perfs','pellicules','mi-juin','20H','longs','loin','chorégraphies','management','vasculaire','abritant'
                   ,'perdition','promus','Auprès','bagages','coteaux','Cage','médicamenteux','douche','olives','moto','internautes',
                   'intervention','symbolise','diffuseur','intérieure','décrochent','tissus','Patinoire','Protestants','pneumonie',
                   'bénéficiez','inscris','Félicitations','prédécesseurs','definition','sachets','Massif','lueurs','bureau','pilier',
                   'respectif','homophobes','Piège','Spaghetti','Cèdre','SUPPORT','désavantage','composites','inaperçu','selection',
                   'bienvenu','Défense','tradition','buffets','Auto-école','INTENDANCE','jour','pale','exhortation','Placements',
                   'boites','Traité','enclin','semoule','raciales','secrets','validée','Parcours','barrage','copains','Flotte',
                   'humus','gisement','rattraper','BANCAIRE','académique','dynasties','Palmer','rangs','gratuit','Rechercher',
                   'envole','jetons','Rosiers','altitude','sentinelles','Guiraud','embarcadère','Pays','vaporisateur','nourrisson',
                   'durcit','psychothérapeute','tirs','coin','usagés','assassinats','diagnostics','Sexualité','réparateur',
                   'ORDRE','surnoms','Célébration','taches','coulissant','sacerdoce','Glossaire','CIRRHOSE','pattes','coffrage',
                   'Total','Galles','commandes','SYSTÈME','cinéastes','glaciale','Court-métrage','Bowling','attrait','Salonique',
                   'inspirations','Conclusions','gonfle','chauffage','Avenir','Objectif','qualitatif','endommagé','Boulonnerie',
                   'FORMULAIRE','affects','serein']
    

In [195]:
y = list(np.zeros(len(Gender_Neutral)))+list(np.ones(len(gender_specific)))
df_train = pd.DataFrame(data = {'Mot':Gender_Neutral+ gender_specific,'label':y})
def built_X_y(result,model):
    X=[]
    for word in list(result['Mot']):
        X.append(model[word])
        
    y = result['label'].values
    return(np.asarray(X),y)
x_train,y_train = built_X_y(df_train,model_cc)

In [199]:
clf = svm.SVC(gamma='scale',class_weight = {1:100,0:1})
clf.fit(x_train, y_train)  

SVC(C=1.0, cache_size=200, class_weight={1: 100, 0: 1}, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='scale', kernel='rbf',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False)

In [202]:
def built_X_test(n,model):
    df = pd.DataFrame(list(model.wv.vocab.items()), columns=['word','count'])
    list_word = list(df['word'])
    shuffle(list_word)
    X_test = []
    label_test = []
    for word in list_word[:n]:
        X_test.append(model[word])
        label_test.append(word)
    return(X_test,label_test)

In [203]:
X_test,label_test = built_X_test(10000,model_cc)
y_pred = clf.predict(X_test)
df_result = pd.DataFrame(data={'word':label_test,'sem':y_pred})

C:\Users\Robin\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  


In [211]:
list(df_result[df_result['sem']==1]['word'])

['mon',
 'voulut',
 'Anja',
 'sangle',
 'casque',
 'katana',
 'Favreau',
 'épée',
 'Bard',
 'corp',
 'aîné',
 'angora',
 'Poulet',
 'Redford',
 'Hassan',
 'ans.',
 'tendon',
 'Hillary',
 'blonds',
 'concubine',
 'moineau',
 'Dürer',
 'Tom',
 'paddock',
 'Pompadour',
 'Ami',
 'bite',
 '1681',
 'Susanne',
 'maligne',
 'dandy',
 'compère',
 'Pine',
 'cuit',
 'dresseur',
 'farcie',
 'recueillie',
 'Serbe',
 'biologiste',
 'diadème',
 'Charlène',
 'second',
 'Thorgal',
 'juive',
 'Britney',
 'Goupil',
 'Willi',
 'proviseur',
 'Révérend',
 'volante',
 'sitcom',
 'garagiste',
 'dossard',
 'ferré',
 'pan',
 'latéral',
 'boucher',
 'miel',
 'fugitif',
 'Kay',
 'Louane',
 'Capucine',
 'curateur',
 'Kiwi',
 'Ryu',
 'truand',
 'trapèze',
 'Mouton',
 'skipper',
 'Hathaway',
 'Broca',
 '4ème',
 'électeur',
 'bouée',
 'Yorkshire',
 'Gareth',
 'épousant',
 '1734',
 '7eme',
 'Gisèle',
 'féline',
 'marine',
 'Cary',
 'atout',
 'Tigre',
 'Fanny',
 'Lukáš',
 'tunique',
 'naïf',
 'pâture',
 'vipère',
 'Kar